This code is reference by:https://github.com/Strata-Scratch/api-youtube

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime

!pip install psycopg2
import psycopg2 as ps

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def get_video_details(video_id):

    #collecting view, like, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    if 'likeCount' in response_video_stats['items'][0]['statistics']:
      like_count = response_video_stats['items'][0]['statistics']['likeCount']
    else:
      like_count = 0
    
    if 'commentCount' in response_video_stats['items'][0]['statistics']:
      comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    else:
      comment_count = 0

    
    url_video_time = 'https://www.googleapis.com/youtube/v3/videos?id='+ video_id +'&part=contentDetails&key='+API_KEY
    response_video_time = requests.get(url_video_time).json()
    if 'duration' in response_video_time['items'][0]['contentDetails']:
      duration = response_video_time['items'][0]['contentDetails']['duration']
    else:
      duration = 0

    url_video_describe = 'https://www.googleapis.com/youtube/v3/videos?part=snippet&id='+video_id+'&key='+ API_KEY
    response_video_describe = requests.get(url_video_describe).json()
    if 'tags' in response_video_describe['items'][0]['snippet']:
      tags = response_video_describe['items'][0]['snippet']['tags']
    else:
      tags =[]
    
    if 'description'  in response_video_describe['items'][0]['snippet']:
      description = response_video_describe['items'][0]['snippet']['description']
    else:
      description = ''
    return view_count, like_count, comment_count, duration, tags, description

In [3]:
def getduration(str):
	duratuion = 0
	if str is not None :
		
		if str.find('PT')>=0:
			str = str.replace('PT', '')
		if str.find('H')>0:
			h = str.find('H')
			hr = str[:h]
			str = str[h+1:]
			if hr.isdigit():
				hour = int(hr)
			else:
				hour = 0
		else:
			hour = 0
		
		if str.find('M')>0:
			min = 0
			m = str.find('M')
			mins = str[:m]
			str = str[m+1:]
			if mins.isdigit():
				min = int(mins)
			else:
				min = 0
		else:
			min=0
		if str.find('S')>0:
			second = 0
			s = str.find('S')
			seconds = str[:s]
			str = str[s+1:]
			if seconds.isdigit():
				second = int(seconds)
			else:
				second = 0
		else:
			second = 0 
		
		duration = float(second/60*1.0)+ min +hour *60
	else:
		duration =0.0
					
	return duration

def yeartoday(upload_data):
	if upload_data != '':
		start_date = datetime.strptime(upload_data, "%Y-%m-%d")
		end_date = datetime.strptime('12/26/2021', "%m/%d/%Y")
		return (end_date-start_date).days
		
	else:
		return 100

In [9]:
def get_videos(df):
    pageToken = ""
    while 1:
        sub_url = 'https://www.googleapis.com/youtube/v3/channels?part=statistics&id=' + CHANNEL_ID +'&key='+ API_KEY
        if 'subscriberCount' in requests.get(sub_url).json()['items'][0]['statistics']:
          subscriber = requests.get(sub_url).json()['items'][0]['statistics']['subscriberCount']
        else:
          subscriber = 0
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
        

        response = requests.get(url).json()
        ct = datetime.now()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count, duration_count, tags, description= get_video_details(video_id)
                duration_count = getduration(duration_count)
                upload_days = yeartoday(upload_date)

                if len(df.loc[df['video_id'] == video_id].index.tolist())==0:
                  
                  df = df.append({"channel_id":CHANNEL_ID ,'video_id':video_id,'video_title':video_title,'video_tags':tags,
                                  'video_description' : description,
                                  "upload_date":upload_date,"view_count":view_count,
                                  "like_count":like_count, "subscriber_count":subscriber,
                                  "comment_count":comment_count, 'duration_count' : duration_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [26]:
df2 = pd.DataFrame(columns=["channel_id","video_id","video_title",'video_tags',
                                'video_description' ,"upload_date","view_count","like_count","subscriber_count","comment_count", 'duration_count']) 


In [25]:
#API_KEY = 'your API key'
#1362
CHANNEL_ID_list = ['UCvmxzBDt6zpM1BhZaGeiV0Q','UC1dVfl5-I98WX3yCy8IJQMg','UC-gjtyoUTzGEwWM5Y1J4J7Q','UCNlfGuzOAKM1sycPuM_QTHg','UCjzHeG1KWoonmf9d5KBvSiw','UCJhjE7wbdYAae1G25m0tHAA','UCQtJT-hZA3FDTDHnQt9Qqsg','UCSJ4gkVC6NrvII8umztf0Ow']

for CHANNEL_ID in CHANNEL_ID_list:
  df2 = get_videos(df2)







In [11]:
API_KEY = 'your API key'
#3116
CHANNEL_ID_list = ['UCL76qU6Tnitr-X-x82WbCGw','UCKHsXi0fY-mvLAAnLq95PPQ','UCtBBEQw4SFYOcrZKzY7quCQ','UCJuMbdKSMThk2RpALASyXVQ','UChyRMEp8zRio9nFnAjvzSnA','UCJIOFQLGwB3GH9K9waxwynQ','UC1XsdqFNRDsiB0ZYem_u4RQ','UCrMI3Ti5xys2eLng2HaMo1A','UCGN5K71CJEmfIVjr-6rjNPA']

for CHANNEL_ID in CHANNEL_ID_list:
  df2 = get_videos(df2)


In [13]:
API_KEY = 'your API key'
#4633
CHANNEL_ID_list = ['UCQINXHZqCU5i06HzxRkujfg','UC_iK_3e3JAP0UzgGQ7NLg9w','UCyD59CI7beJDU493glZpxgA','UCz9_4daWw-uWuqeB6_IkhMg','UCi8wqezBudeAiTdKOX571ug','UCr1tljCdi0qH0WSy5z0xp2g']

for CHANNEL_ID in CHANNEL_ID_list:
  df2 = get_videos(df2)

In [15]:
API_KEY = 'your API key'
#6214
CHANNEL_ID_list = ['UC2fVSthyWxWSjsiEAHPzriQ','UCxONCRXEhzv0cRbWCKybOsw','UCWzZ5TIGoZ6o-KtbGCyhnhg','UC0fiLCwTmAukotCXYnqfj0A']

for CHANNEL_ID in CHANNEL_ID_list:
  df2 = get_videos(df2)

In [17]:
API_KEY = 'your API key'
#1362
CHANNEL_ID_list = ['UCwkTfp14Sj7o6q9_8ADJpnA','UCZyyXrEF2WCZbI653PFNBbA','UC7G09-8q4gdyL4ylpw3TfSw','UCchwNR9bTzVNsrjF7mED0gQ','UCLMSs5YDNPZ1Pnm1zD_T5vg','UC4SnPoPC5g4LKsRsLVNIhmw']

for CHANNEL_ID in CHANNEL_ID_list:
  df2 = get_videos(df2)

In [27]:
df2

,channel_id,video_id,video_title,video_tags,video_description,upload_date,view_count,like_count,subscriber_count,comment_count,duration_count


In [21]:
df2.to_csv('youtube_playlist.csv')


In [82]:
["channel_id","video_id","video_title",'video_tags',
                                'video_description' ,"upload_date","view_count","like_count","subscriber_count","comment_count", 'duration_count']) 

1

In [37]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    channel_id VARCHAR(255) NOT NULL,
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    Video_tags TEXT NOT NULL,
                    video_description TEXR NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    subscriber_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL,
                    duration_count FLOAT NOT NULL
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, channel_id,video_id, video_title, Video_tags, video_description, upload_date,
                        view_count, like_count, subscriber_count,comment_count, duration_count):
    insert_into_videos = ("""INSERT INTO videos (channel_id,video_id, video_title, Video_tags, video_description, upload_date,
                        view_count, like_count, subscriber_count,comment_count, duration_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (channel_id,video_id, video_title, Video_tags, video_description, upload_date,
                        view_count, like_count, subscriber_count,comment_count, duration_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, channel_id,video_id, video_title, Video_tags, video_description, upload_date,
                        view_count, like_count, subscriber_count,comment_count, duration_count):
    query = ("""UPDATE videos
            SET channel_id = %s,
                video_title = %s,
                video_tags = %s,
                video_description = %s,
                view_count = %s,
                like_count = %s,
                subscriber_count = %s,
                comment_count = %s,
                duration_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (channel_id,video_id, video_title, Video_tags, video_description, upload_date,
                        view_count, like_count, subscriber_count,comment_count, duration_count)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['channel_id'], row['video_id'], row['video_title'],row['video_tags'], row['video_description'], row['upload_date'], row['view_count']
                          , row['like_count'], row['dislike_count'], row['comment_count'],row['duration_count'])


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=["channel_id","video_id","video_title",'video_tags',
                                'video_description' ,"upload_date","view_count","like_count","subscriber_count","comment_count", 'duration_count']) 
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['channel_id'], row['video_id'], row['video_title'],row['video_tags'], row['video_description'], row['upload_date'], row['view_count']
                          , row['like_count'], row['dislike_count'], row['comment_count'],row['duration_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [22]:
#Main

#database credentials
host_name = ''
dbname = ''
port = '3306'
username = '' 
password = ''
conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()